In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D,Dropout,Dense,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import load_img,array_to_img, ImageDataGenerator
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
import cv2
import time
encoder = OneHotEncoder()

In [3]:
training_batch_size = 130
validation_batch_size = 1657
image_size = 128

In [18]:
def preprocessTwo():
    print("Preprocessing Data...")
    train = pd.read_csv("../Corona Chest X-Rays/Data_information/Chest_xray_Corona_Metadata.csv")
    test = pd.read_csv("coronaClassifier/Corona Chest X-Rays/Chest_xray-Corona_Metadata_test.csv")
    train_directory = 'coronaClassifier/Corona Chest X-Rays/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'
    test_directory = 'coronaClassifier/Corona Chest X-Rays/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'
   
    train['image'] = [s + ".jpeg" for s in train["X_ray_image_name"]]
    train['target_string'] = train['Label'].astype(str)
    train_df, validate_df = train_test_split(train, test_size=0.05, random_state=42)
    
    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    
    train_generator = train_datagen.flow_from_dataframe(
        train_df,
        directory = train_directory,
        x_col = 'image',
        y_col = 'target',
        target_size = (image_size,image_size),
        shuffle = True,
        class_mode = 'raw')
    
    validation_datagen = ImageDataGenerator(rescale=1./255)
    
    validation_generator = validation_datagen.flow_from_dataframe(
        validate_df, 
        directory = train_directory,
        x_col = 'image',
        y_col = 'target',
        target_size = (image_size,image_size),
        shuffle = True,
        class_mode = 'raw')
    
    print("Finished Preprocessing")
    
    return train_generator, validation_generator

In [4]:
class MoleNet:
    def build(image_size):
        model = Sequential() 
        
        model.add(Conv2D(32,(3,3), activation = 'relu', input_shape = (image_size,image_size,3))) 
        model.add(MaxPool2D((2,2))) 
        model.add(Dropout(rate=0.5))
        model.add(Conv2D(64,(3,3), activation = 'relu')) 
        model.add(MaxPool2D((2,2))) 
        model.add(Conv2D(128,(3,3), activation = 'relu')) 
        model.add(MaxPool2D((2,2))) 
        model.add(Conv2D(256,(3,3), activation = 'relu')) 
        model.add(MaxPool2D((2,2))) 
        model.add(Flatten()) 
        model.add(Dense(256, activation = 'relu'))
        model.add(Dropout(rate=0.25))
        model.add(Dense(512, activation = 'relu'))
        model.add(Dropout(rate=0.25))
        model.add(Dense(256, activation = 'relu'))
        model.add(Dropout(rate=0.25))
        model.add(Dense(1, activation = 'sigmoid'))


        model.summary()
        return model

In [5]:
def trainModel(train_generator, validation_generator, training_batch_size, validation_batch_size, number_of_epochs,image_size):
    print("Training Model...")
    
    model = MoleNet.build(image_size)
    model.compile(loss='binary_crossentropy',optimizer='Adadelta',metrics=['accuracy'])
    model.fit_generator(train_generator, steps_per_epoch=train_generator.samples//training_batch_size, epochs=number_of_epochs, validation_data=validation_generator, validation_steps = validation_generator.samples//validation_batch_size,class_weight = class_weight)
    print("Model Trained")
    return model
    

In [6]:
def testModel(model,image_size):
    test_csv = pd.read_csv('coronaClassifier/Corona Chest X-Rays/Chest_xray-Corona_Metadata_test.csv')

    testImages = []
    image_name=[]

    for index,row in test_csv.iterrows():
        print(index)
        testImages.append(cv2.resize(cv2.imread('coronaClassifier/Corona Chest X-Rays/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'+row['X_ray_image_name']+'.jpeg'),(image_size,image_size)))
        image_name.append(row['image_name'])

    testImages = np.array(testImages)
    
    pred = model.predict(testImages)
    pred = pred.ravel()
    print(accuracy_score(y_test,pred))
    
    print(len(pred))
    print(len(image_name))

In [19]:
train_generator, validation_generator = preprocessTwo()

Preprocessing Data...


FileNotFoundError: [Errno 2] File ../Corona Chest X-Rays/Data_information/Chest_xray_Corona_Metadata.csv does not exist: '../Corona Chest X-Rays/Data_information/Chest_xray_Corona_Metadata.csv'

In [ ]:
validation_batch_size = 1657
number_of_epochs = 3
model = trainModel(train_generator, validation_generator,training_batch_size, validation_batch_size, number_of_epochs,image_size)


In [ ]:
testModel(model,image_size)